https://github.com/huggingface/transformers/issues/17971

In [1]:

from PIL import Image
import pandas as pd

def file_check(x):
    try:
        img = Image.open(f'images/{x}.jpg')
        return img.mode
    except:
        return False
    
df = pd.read_csv('dataset_curso.csv')
df['has_image'] = df['id'].apply(file_check)


In [2]:
df.groupby('has_image').count()

,clean_title,created_utc,id,image_url,linked_submission_id,num_comments,score,upvote_ratio,2_way_label
has_image,,,,,,,,,
False,16586,16586,16586,16451,14564,2022,16586,2022,16586
CMYK,1,1,1,1,1,0,1,0,1
L,32,32,32,32,10,22,32,22,32
P,2699,2699,2699,2699,96,2603,2699,2603,2699
RGB,37073,37073,37073,37073,2099,34974,37073,34974,37073
RGBA,9,9,9,9,9,0,9,0,9


In [3]:

image_dataset = df[df['has_image'] == 'RGB']

image_dataset['image'] = image_dataset['id'].apply(lambda x: f'images/{x}.jpg')
image_dataset['labels'] = image_dataset['2_way_label']
df = image_dataset[['image', 'labels']]

df.describe()

/var/folders/g_/0lqz2qlx7cx96d901w35swh40000gn/T/ipykernel_55878/1001678944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_dataset['image'] = image_dataset['id'].apply(lambda x: f'images/{x}.jpg')
/var/folders/g_/0lqz2qlx7cx96d901w35swh40000gn/T/ipykernel_55878/1001678944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_dataset['labels'] = image_dataset['2_way_label']


,labels
count,37073.000000
mean,0.552531
std,0.497239
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [4]:
df['image'].describe()

count                 37073
unique                37073
top       images/239vnh.jpg
freq                      1
Name: image, dtype: object

In [5]:
from datasets import Dataset, Image

image_dataset = Dataset.from_pandas(df) \
    .cast_column('image', Image()) \
    .train_test_split(test_size=0.2)
image_dataset

/Users/dany/.local/share/virtualenvs/fake_news-6ZUGHfY-/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['image', 'labels', '__index_level_0__'],
        num_rows: 29658
    })
    test: Dataset({
        features: ['image', 'labels', '__index_level_0__'],
        num_rows: 7415
    })
})

In [6]:
from transformers import AutoImageProcessor

model_id = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(model_id)

In [7]:
def transform(example_batch):
  inputs = image_processor([x for x in example_batch["image"]], return_tensors="pt")
  inputs["labels"] = example_batch["labels"]
  return inputs

In [8]:
prepared_ds = image_dataset.with_transform(transform)

In [9]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch])
  }

In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(prediction):
  return metric.compute(
    predictions=np.argmax(prediction.predictions, axis=1), 
    references=prediction.label_ids
)

/var/folders/g_/0lqz2qlx7cx96d901w35swh40000gn/T/ipykernel_55878/995733927.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [11]:
from transformers import ViTForImageClassification


model = ViTForImageClassification.from_pretrained(
    model_id,
    num_labels=2,
    # id2label={str(i): c for i, c in enumerate(labels)},
    # label2id={c: str(i) for i,c in enumerate(labels)}
)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./image_model",
    evaluation_strategy="steps",
    num_train_epochs=4,
    learning_rate=2e-4,
    load_best_model_at_end=True,
    remove_unused_columns=False,
)

In [13]:
image_dataset["train"].features

{'image': Image(decode=True, id=None),
 'labels': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=image_processor
)

In [15]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)

  3%|▎         | 500/14832 [16:18<6:40:03,  1.67s/it] 

{'loss': 0.6897, 'learning_rate': 0.00019325782092772384, 'epoch': 0.13}


                                                     
  3%|▎         | 500/14832 [26:58<6:40:03,  1.67s/it]

{'eval_loss': 0.6864238977432251, 'eval_accuracy': 0.5611598111935266, 'eval_runtime': 639.9837, 'eval_samples_per_second': 11.586, 'eval_steps_per_second': 1.448, 'epoch': 0.13}


  7%|▋         | 1000/14832 [43:24<7:05:12,  1.84s/it]  

...

{'loss': 0.6639, 'learning_rate': 4.47680690399137e-06, 'epoch': 3.91}


                                                        
 98%|█████████▊| 14500/14832 [11:47:10<09:57,  1.80s/it]

{'eval_loss': 0.6658162474632263, 'eval_accuracy': 0.6006743088334457, 'eval_runtime': 499.1411, 'eval_samples_per_second': 14.856, 'eval_steps_per_second': 1.857, 'epoch': 3.91}


100%|██████████| 14832/14832 [11:57:30<00:00,  2.90s/it]    


{'train_runtime': 43050.8307, 'train_samples_per_second': 2.756, 'train_steps_per_second': 0.345, 'train_loss': 0.6794167398249061, 'epoch': 4.0}
***** train metrics *****
  epoch                    =         4.0
  train_loss               =      0.6794
  train_runtime            = 11:57:30.83
  train_samples_per_second =       2.756
  train_steps_per_second   =       0.345


In [17]:
metrics = trainer.evaluate(prepared_ds["test"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

100%|██████████| 927/927 [09:26<00:00,  1.64it/s]

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.6007
  eval_loss               =     0.6658
  eval_runtime            = 0:09:33.07
  eval_samples_per_second =     12.939
  eval_steps_per_second   =      1.618
